In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras import backend 

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tensorflow as tf
import numpy as np
from keras.utils.vis_utils import plot_model
import visualkeras
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
from scipy.io import loadmat
import librosa
import librosa.display
import pickle
import numpy as np
import mne
import os
import cv2
import random
import math
import shutil

In [ ]:
patient_numbers = range(1, 33)

In [ ]:
def create_or_overwrite_dir(addr):
    if os.path.exists(addr):
        shutil.rmtree(addr)
    os.makedirs(addr)

In [ ]:
for patient_number in patient_numbers:
    print(f"####################{patient_number}##########################")
    
    src_list = [f"DEAP_Dataset/PSDEAPTrData/t_data_s{patient_number}/classification_files/original/",
                f"DEAP_Dataset/PSDEAPTrData/t_data_s{patient_number}/classification_files/origGen/",
               f"DEAP_Dataset/PSDEAPTrData/t_data_s{patient_number}/classification_files/origNorm/"]

    
    dst_list = [f"NIQE/patient{patient_number}/original/", 
                f"NIQE/patient{patient_number}/generated/",
                f"NIQE/patient{patient_number}/noise/"]

    
    for index, src in enumerate(src_list):
        dst = dst_list[index]
        
        create_or_overwrite_dir(dst)
        
        files = os.listdir(src)
        
        for file in files:
            with open(src + file, 'rb') as handle:
                loaded_data = pickle.load(handle)
                data = loaded_data["data"]
            
            plt.figure(figsize=(9,12))
            plt.axis('off')
        
            plt.imshow(data)
            plt.savefig(f"{dst}{file}.png", dpi=600, transparent=True)
            
            plt.close()
        
        
    
    
    

In [ ]:
data_list = dict()

generated_scores = list()
original_scores = list()
noise_scores = list()

patients = [2, 3, 4, 7, 10, 19 ,28 , 30]

for patient in patients:
    data = loadmat(f"NIQE/scores/patient{patient}.mat")
    generated_scores.append(data["generated_NIQE"][0])
    original_scores.append(data["original_NIQE"][0])
    noise_scores.append(data["noise_NIQE"][0])

    

original_scores = np.array(original_scores)
original_scores = original_scores.reshape(original_scores.shape[0]*original_scores.shape[1])
original_scores = original_scores.tolist()
    
generated_scores = np.array(generated_scores)
generated_scores = generated_scores.reshape(generated_scores.shape[0]*generated_scores.shape[1])
generated_scores = generated_scores.tolist()

noise_scores = np.array(noise_scores)
noise_scores = noise_scores.reshape(noise_scores.shape[0]*noise_scores.shape[1])
noise_scores = noise_scores.tolist()


In [ ]:
print(annots["generated_NIQE"].shape)

In [ ]:
original_mean_of_subjects = list()
original_std_of_subjects = list()
original_range_of_subjects = list()
original_mse_of_subjects = list()

generated_mean_of_subjects = list()
generated_std_of_subjects = list()
generated_range_of_subjects = list()
generated_mse_of_subjects = list()

noise_mean_of_subjects = list()
noise_std_of_subjects = list()
noise_range_of_subjects = list()
noise_mse_of_subjects = list()

for patient in range(1, 33):
    annots = loadmat(f"NIQE/scores/patient{patient}.mat")

    original_mean_of_subjects.append(annots["original_NIQE"].mean())
    original_std_of_subjects.append(annots["original_NIQE"].std())
    original_range_of_subjects.append(annots["original_NIQE"].max()-annots["original_NIQE"].min())
    original_mse_of_subjects.append(np.square(np.subtract(annots["original_NIQE"],annots["original_NIQE"])).mean())
    #######################################################################################
    generated_mean_of_subjects.append(annots["generated_NIQE"].mean())
    generated_std_of_subjects.append(annots["generated_NIQE"].std())
    generated_range_of_subjects.append(annots["generated_NIQE"].max()-annots["generated_NIQE"].min())
    generated_mse_of_subjects.append(np.square(np.subtract(np.append(annots["original_NIQE"], annots["original_NIQE"]), annots["generated_NIQE"])).mean())
    ######################################################################################
    noise_mean_of_subjects.append(annots["noise_NIQE"].mean())
    noise_std_of_subjects.append(annots["noise_NIQE"].std())
    noise_range_of_subjects.append(annots["noise_NIQE"].max()-annots["noise_NIQE"].min())
    noise_mse_of_subjects.append(np.square(np.subtract(annots["original_NIQE"], annots["noise_NIQE"])).mean())
    
print(np.mean(original_mean_of_subjects))
print(np.mean(original_std_of_subjects))
print(np.mean(original_range_of_subjects))
print(np.mean(original_mse_of_subjects))
print("###############################################################")
print(np.mean(generated_mean_of_subjects))
print(np.mean(generated_std_of_subjects))
print(np.mean(generated_range_of_subjects))
print(np.mean(generated_mse_of_subjects))
print("###############################################################")
print(np.mean(noise_mean_of_subjects))
print(np.mean(noise_std_of_subjects))
print(np.mean(noise_range_of_subjects))
print(np.mean(noise_mse_of_subjects))

In [ ]:
y = list()

for subject in patients:
    for sample in range(0, 64): 
        y.append(f"Subject {subject}")

fig = go.Figure()
fig.add_trace(go.Box(
    x=original_scores,
    y=y,
    name='Authentic',
    marker_color='#3D9970'
))

y = list()

for subject in patients:
    for sample in range(0, 128): 
        y.append(f"Subject {subject}")

        
fig.add_trace(go.Box(
    x=generated_scores,
    y=y,
    name='GAN Generated',
    marker_color='#FF4136'
))

y = list()

for subject in patients:
    for sample in range(0, 64): 
        y.append(f"Subject {subject}")

        
fig.add_trace(go.Box(
    x=noise_scores,
    y=y,
    name='Noise Generated',
    marker_color='#FF851B'
))

fig.update_layout(
    xaxis=dict(title='NIQE Score', zeroline=False),
    boxmode='group'
)

fig.update_traces(orientation='h') # horizontal box plots
fig.show()